In [3]:
import os
import random
# XML 다루는 라이브러리
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as data

%matplotlib inline

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [28]:
# 학습 및 검증용 화상 데이터, 어노테이션 데이터 파일 경로 리스트 작성
def make_datapath_list(rootpath):
    img_file_path = os.path.join(rootpath, 'JPEGImages', '%s.jpg') # %연산 시 3번째 arg형태로 파일명이 됨
    annotation_path = os.path.join(rootpath, 'Annotations', '%s.xml')

    train_file_id = os.path.join(rootpath + 'ImageSets/Main/train.txt')
    val_file_id = os.path.join(rootpath + 'ImageSets/Main/val.txt')

    train_img_list = []
    train_annotation_list = []

    for line in open(train_file_id): # txt파일을 한줄씩 읽어옴
        file_id = line.strip() 
        img_file_name = (img_file_path % file_id)  # 画像のパス
        anno_file_name = (annotation_path % file_id)  # アノテーションのパス
        train_img_list.append(img_file_name)  # リストに追加
        train_annotation_list.append(anno_file_name)  # リストに追加

    val_img_list = []
    val_annotation_list = []

    for line in open(val_file_id): # txt파일을 한줄씩 읽어옴
        file_id = line.strip()  
        img_file_name = (img_file_path % file_id)  # 画像のパス
        anno_file_name = (annotation_path % file_id)  # アノテーションのパス
        val_img_list.append(img_file_name)  # リストに追加
        val_annotation_list.append(anno_file_name)  # リストに追加

    return train_img_list, train_annotation_list, val_img_list, val_annotation_list

In [29]:
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

print(train_img_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg


In [34]:
class Anno_xml2list(object): 
    def __init__(self, classes):

        self.classes = classes

    def __call__(self, xml_path, width, height):
        # 물체의 어노테이션 데이터를 저장한 리스트. 이미지에 존재하는 물체 수만큼 len을 가짐.
        ret = [] #ret : [[xmin, ymin, xmax, ymax, label_ind], ... ]

        # xml파일로드
        xml = ET.parse(xml_path).getroot()

        # xml에서 <object>의 수만큼 반복
        for obj in xml.iter('object'):

            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                continue

            # 한 물체의 바운딩 박스 어노테이션 정보를 저장하는 리스트
            bndbox = []

            name = obj.find('name').text.lower().strip()  # 물체이름
            bbox = obj.find('bndbox')  # 바운딩박스 정보

            pts = ['xmin', 'ymin', 'xmax', 'ymax']

            for pt in (pts):
                # VOC데이터셋은 원점이 (1,1)이므로 빼줌
                cur_pixel = int(bbox.find(pt).text) - 1

                # 정규화
                if pt == 'xmin' or pt == 'xmax':  # x의 경우 width로 나눔
                    cur_pixel /= width
                else:  # y는 높이인 height로 나눔
                    cur_pixel /= height

                bndbox.append(cur_pixel)

            # 어노테이션의 class명에 해당하는 index번호 추가
            label_idx = self.classes.index(name)
            bndbox.append(label_idx)

            ret += [bndbox]

        return np.array(ret) #[[xmin, ymin, xmax, ymax, label_ind], ... ]

In [39]:
val_anno_list[1], val_img_list[1]

('./data/VOCdevkit/VOC2012/Annotations/2008_000003.xml',
 './data/VOCdevkit/VOC2012/JPEGImages/2008_000003.jpg')

In [37]:
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

transform_anno = Anno_xml2list(voc_classes)

ind = 1
image_file_path = val_img_list[ind]
img = cv2.imread(image_file_path)  # [높이][폭][RGB]
height, width, channels = img.shape  # 이미지 크기

# 어노테이션 리스트로 표시
transform_anno(val_anno_list[ind], width, height) # 해당 파일엔 object가 2개 있음

array([[ 0.09      ,  0.03003003,  0.998     ,  0.996997  , 18.        ],
       [ 0.122     ,  0.56756757,  0.164     ,  0.72672673, 14.        ]])